##### Copyright 2019 The TensorFlow Authors.

In [5]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import datetime
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import StandardScaler
import numpy as np # for math and arrays
import pandas as pd
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler



data = pd.read_csv("C:/Users/reifv/root/Heidelberg Master/vs_codes/tf_nn_hparams_tuner_blueprint/data/diabetes_data.csv", delimiter=";")
data = pd.concat([data, pd.get_dummies(data['gender'], prefix='Label')], axis=1)
data.pop("gender")
train_dataset, temp_test_dataset =  train_test_split(data, test_size=0.3,random_state=42)
test_dataset, valid_dataset =  train_test_split(temp_test_dataset, test_size=0.5,random_state=42)
train_labels = train_dataset.pop('class')
test_labels = test_dataset.pop('class')
valid_labels = valid_dataset.pop('class')
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
normed_train_data = pd.DataFrame(StandardScaler().fit_transform(train_dataset), columns=train_dataset.columns, index=train_dataset.index)
normed_test_data = pd.DataFrame(StandardScaler().fit_transform(test_dataset), columns=test_dataset.columns, index=test_dataset.index)
normed_valid_data = pd.DataFrame(StandardScaler().fit_transform(valid_dataset), columns=valid_dataset.columns, index=valid_dataset.index)
x_train, y_train, x_valid, y_valid = normed_train_data, train_labels, normed_valid_data, valid_labels

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([5]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_LR = hp.HParam('lr', hp.Discrete([0.0001]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([2]))
METRIC_ACCURACY = 'accuracy'

# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32,64,128,256,1000,2000]))
# HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1,2,3,4,5]))
# # HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
# HP_LR = hp.HParam('lr', hp.Discrete([0.1,0.01,0.001,0.0001,]))
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1,32,256]))
# METRIC_ACCURACY = 'accuracy'

# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32,64,128,256,1000]))
# HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1,2,3,4]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
# HP_LR = hp.HParam('lr', hp.Discrete([0.001,0.0001,]))
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1,2,16,32,64]))
# METRIC_ACCURACY = 'accuracy'

log_name = 'logs_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'/hparam_tuning'

with tf.summary.create_file_writer(log_name).as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_NUM_LAYERS, HP_OPTIMIZER, HP_LR, HP_BATCH_SIZE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def helper_fct_return_optimizer_w_learn_rate(opt_name:str,lr:float):
  if opt_name == "sgd":
    return tf.keras.optimizers.SGD(learning_rate=lr)
  if opt_name == "adam":
    return tf.keras.optimizers.Adam(learning_rate=lr)
  return "error"

def train_test_model(hparams,run_dir):
  body = tf.keras.models.Sequential()
  for _ in range(int(hparams[HP_NUM_LAYERS])):
    body.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS]))
      # tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
  body.add(tf.keras.layers.Dense(1))
  model = body
  model.compile(
      optimizer=helper_fct_return_optimizer_w_learn_rate(hparams[HP_OPTIMIZER],hparams[HP_LR],),
      loss='BinaryCrossentropy', metrics=['accuracy'],)
  model.fit(x_train, y_train,validation_data=(x_valid,y_valid),epochs=1, shuffle=True,verbose=False, callbacks=[ tf.keras.callbacks.TensorBoard(log_dir=run_dir+'_'+str(hparams[HP_NUM_LAYERS])+'layers_'+str(hparams[HP_NUM_UNITS])+'nodes_'+hparams[HP_OPTIMIZER]+str(hparams[HP_LR])+'_'+str(hparams[HP_BATCH_SIZE])+'batchsize_', histogram_freq=1)],batch_size=(hparams[HP_BATCH_SIZE])) 
  _, accuracy = model.evaluate(x_valid, y_valid,verbose=False)
  return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams,run_dir)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0
for num_units in tqdm(HP_NUM_UNITS.domain.values):
  for num_layers in HP_NUM_LAYERS.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      for lr in HP_LR.domain.values:   
        for batch_size in HP_BATCH_SIZE.domain.values:
          hparams = {HP_NUM_UNITS: num_units,HP_NUM_LAYERS: num_layers, HP_OPTIMIZER: optimizer, HP_LR: lr, HP_BATCH_SIZE: batch_size }
          run_name = "run-%d" % session_num
          run(log_name+ run_name, hparams )
          session_num += 1
          # problem anschauen dass wenn script druch cpu und memory immer voll

# tb in browser:
print("tensorboard --logdir "+log_name[:20]+" --port "+log_name[14:18])


100%|██████████| 1/1 [00:03<00:00,  3.72s/it]

tensorboard --logdir logs_20220606-130014 --port 1300


In [4]:
print("tensorboard --logdir "+log_name[:20]+" --port "+log_name[14:18])

tensorboard --logdir logs_20220606-124205 --port 1242
